Kod pozwala na analizę wpływu wartości odcięcia pędu dla neutrinów na stosunek sygnału do tła. Do analizy potrzebne są dane uzyskane za pomoca skryptu Eksport_danych_z_root.ipynb z modyfikacją, postaci usunięcia kodu PDG neutronów (2112) z listy cząstek neutralnych.

In [ ]:
import csv
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


In [ ]:
# Funkcja do wczytywania danych
def loading_data_restructured(file):
    with open(file, 'r', encoding='utf-8') as f:
        all_events = []
        multiplicities = []

        reader = csv.reader(f)
        header = next(reader)  # Pomijamy nagłówek

        for row in reader:
            event_num = int(row[0])
            interaction_type = row[1]
            particles = []

            for i in range(3, len(row), 5):
                try:
                    pid = int(row[i])
                    px = float(row[i+1])
                    py = float(row[i+2])
                    pz = float(row[i+3])
                    E  = float(row[i+4])
                    p = np.array([px, py, pz])
                    M = np.sqrt(max(E**2 - np.dot(p, p), 0))
                    particles.append([pid, px, py, pz, E, M])
                except (ValueError, IndexError):
                    continue

            if not particles:
                continue

            multiplicities.append(len(particles))

            total_px = sum(p[1] for p in particles)
            total_py = sum(p[2] for p in particles)
            total_pz = sum(p[3] for p in particles)
            total_E  = sum(p[4] for p in particles)
            total_p  = np.array([total_px, total_py, total_pz])
            total_M  = np.sqrt(max(total_E**2 - np.dot(total_p, total_p), 0))

            event = [event_num, interaction_type, total_px, total_py, total_pz, total_E, total_M]
            for p in particles:
                event.extend(p)

            all_events.append(event)

    return all_events, multiplicities

In [ ]:
#Funkcja przepisuje event otrzymany poprzednio z nowym warunkiem, w postaci pędu neutronu większego od wartości pędu odcięcia
def save_filtered_events(events, filename, condition_func, neutron_momentum_cut=0.0):
    selected_events = []

    for event in events:
        filtered_particles = []

        # Kopiujemy nagłówek (7 pierwszych kolumn)
        new_event = event[:7]

        for i in range(7, len(event), 6):
            try:
                pid = int(event[i])
                px = float(event[i+1])
                py = float(event[i+2])
                pz = float(event[i+3])
                p = np.linalg.norm([px, py, pz])

                if pid == 2112 and p < neutron_momentum_cut:
                    continue  # Pomijamy neutron o małym pędzie

                filtered_particles.extend(event[i:i+6])
            except (ValueError, IndexError):
                continue

        # Jeśli po filtracji mamy dokładnie 4 cząstki
        if len(filtered_particles) == 24:
            new_event.extend(filtered_particles)
            pids = [int(filtered_particles[j]) for j in range(0, 24, 6)]
            if condition_func(pids):
                selected_events.append(new_event)
    
    return selected_events


def one_proton_three_charged_pions(pids):
    # Liczby typów cząstek
    proton_count = pids.count(2212)
    pion_count = pids.count(211) + pids.count(-211)

    # Sprawdzenie, czy tylko dozwolone cząstki są obecne
    allowed_pids = {2212, 211, -211}
    only_allowed = all(pid in allowed_pids for pid in pids)

    return proton_count == 1 and pion_count == 3 and only_allowed

# Funkcja tworzy zbiory zdarzeń NC i CC o końcowej konfiguracji cząstek 3 piony + proton
def analyze_events(events):
    events_proton_3pions_NC = []
    events_proton_3pions_CC = []

    for event in events:
        interaction_type = event[1]
        particles = []

        i = 7
        while i < len(event):
            pid = int(event[i])
            particles.append(pid)
            i += 6

        if particles.count(2212) == 1:
            pion_count = particles.count(211) + particles.count(-211)
            if pion_count == 3 and len(particles) == 4:
                if interaction_type == 'NC':
                    events_proton_3pions_NC.append(event)
                elif interaction_type == 'CC':
                    events_proton_3pions_CC.append(event)

    return events_proton_3pions_NC, events_proton_3pions_CC


Następnie tworzymy funkcję analizującą wpływ wartości pędu odcięcia na stosunek sygnału do tła

In [ ]:

def analyze_cut_effect(events, condition_func, cut_values):
    ratios = []
    nc_counts = []
    cc_counts = []

    for cut in cut_values:
        filtered = save_filtered_events(events, 'tmp.csv', condition_func, neutron_momentum_cut=cut)
        events_NC, events_CC = analyze_events(filtered)
        nc = len(events_NC)
        cc = len(events_CC)
        nc_counts.append(nc)
        cc_counts.append(cc)
        ratio = cc / nc if nc > 0 else 0
        ratios.append(ratio)
        print(f"Cut: {cut:.2f} GeV → NC: {nc}, CC: {cc}, CC/NC: {ratio:.4f}")

    return cut_values, ratios, nc_counts, cc_counts

def plot_cut_vs_ratio(cuts, ratios):
    plt.figure(figsize=(8,6))
    plt.plot(cuts, ratios, marker='o', linestyle='-', color='blue')
    plt.xlabel('Pęd odcięcia neutronów [GeV]')
    plt.ylabel('Stosunek sygnał / tło')
    plt.title('Efekt wartości odcięcia pędu neutronów na stosunek sygnał / tło')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

Wczytujemy pliki oraz rysujemy wykres.

In [ ]:
cuts = np.arange(0.0, 1.1, 0.1) # Wartości cięć od 0 do 1,1 GeV z odstępami 0,1

files = ['nuTau_data_NC_neutrons_r.csv', 'nuTau_data_CC_neutrons_r.csv']

all_events = []
all_multiplicities = []
#Wczytujemy dane
for f in files:
    events, mult = loading_data_restructured(f)
    all_events.extend(events)
    all_multiplicities.extend(mult)

print(f"Załadowano łącznie {len(all_events)} zdarzeń.")

cut_values, cc_nc_ratios, nc_counts, cc_counts = analyze_cut_effect(
    all_events,
    one_proton_three_charged_pions,
    cuts
)
# Rysujemy wykres
plot_cut_vs_ratio(cut_values, cc_nc_ratios)

W taki sposób otrzymujemy wykres:
![Wpływ pędu odcięcia neutronów na stosunek sygnał do tła](images/Neutron_analiza.png) 

Następnie możemy narysować rozkład pędów neutronów oraz zobaczyć, jaki procent zdarzeń o końcowej konfiguracji cząstek 3 piony + proton zawiera neutrony.

In [ ]:
# Rozkład pędu neutronów 
def plot_neutron_momentum_distribution_normalized(events):
    neutron_momenta_NC = []
    neutron_momenta_CC = []

    for event in events:
        interaction_type = event[1]
        i = 7
        while i < len(event):
            pid = int(event[i])
            px = float(event[i+1])
            py = float(event[i+2])
            pz = float(event[i+3])
            if pid == 2112:  # neutron
                p = np.linalg.norm([px, py, pz])
                if interaction_type == "NC":
                    neutron_momenta_NC.append(p)
                elif interaction_type == "CC":
                    neutron_momenta_CC.append(p)
            i += 6

    plt.figure(figsize=(10,6))
    bins = np.linspace(0, max(neutron_momenta_NC + neutron_momenta_CC + [0.1]), 40)
    plt.hist(neutron_momenta_NC, bins=bins, density=True, alpha=0.6, label="NC", color="green", edgecolor="black")
    plt.hist(neutron_momenta_CC, bins=bins, density=True, alpha=0.6, label="CC", color="orange", edgecolor="black")
    plt.xlim(0, 6)
    plt.xlabel("Pęd neutronu [GeV]")
    plt.ylabel("Gęstość prawdopodobieństwa")
    plt.title("Znormalizowany rozkład pędu neutronów (NC vs CC)")
    plt.legend()
    plt.grid(linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_neutron_presence_in_proton3pions(events):
    nc_zero, nc_nonzero = 0, 0
    cc_zero, cc_nonzero = 0, 0

    for event in events:
        interaction_type = event[1]
        particles = []
        neutrons = 0

        i = 7
        while i < len(event):
            pid = int(event[i])
            if pid == 2112:
                neutrons += 1
            else:
                particles.append(pid)
            i += 6

        # sprawdzamy czy po usunięciu neutronów to 1 proton + 3 piony
        if one_proton_three_charged_pions(particles):
            if interaction_type == "NC":
                if neutrons == 0:
                    nc_zero += 1
                else:
                    nc_nonzero += 1
            elif interaction_type == "CC":
                if neutrons == 0:
                    cc_zero += 1
                else:
                    cc_nonzero += 1

    # obliczamy procenty
    nc_total = nc_zero + nc_nonzero
    cc_total = cc_zero + cc_nonzero
    nc_zero_pct = (nc_zero / nc_total * 100) if nc_total > 0 else 0
    nc_nonzero_pct = (nc_nonzero / nc_total * 100) if nc_total > 0 else 0
    cc_zero_pct = (cc_zero / cc_total * 100) if cc_total > 0 else 0
    cc_nonzero_pct = (cc_nonzero / cc_total * 100) if cc_total > 0 else 0

    # dane do wykresu
    labels = ["NC", "CC"]
    zero_neutrons = [nc_zero_pct, cc_zero_pct]
    nonzero_neutrons = [nc_nonzero_pct, cc_nonzero_pct]

    # wykres skumulowany
    plt.figure(figsize=(8,6))
    plt.bar(labels, zero_neutrons, label="0 neutronów (czyste 1p+3π)", color="skyblue", edgecolor="black")
    plt.bar(labels, nonzero_neutrons, bottom=zero_neutrons, label="≥1 neutron", color="salmon", edgecolor="black")

    plt.ylabel("Procent zdarzeń [%]")
    plt.title("Obecność neutronów w zdarzeniach bazowych 1p+3π (NC vs CC)")
    plt.legend()
    plt.ylim(0, 100)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_neutron_momentum_distribution_normalized(all_events)
plot_neutron_presence_in_proton3pions(all_events)

W taki sposób otrzymujemy poniższe rysunki.
![Rozkład pędów neutronów](images/Neutron_ped.png)
![Procent zdarzeń CC/NC typu 3pi+p z neutronami](images/Neutron_krot.png)